# Data Cleaning

We start with three datasets, one for each oil future. We want them to be in a combined dataset, with all the date indices, and we keep the columns representing the absolute price (Price), the percentage change in price (Change %), and the volume traded (Vol.)

These columns are renamed to Change_{Future}, Price_{Future}, and Vol{Future}, with Future representing the name of the oil future. 

In [5]:
import pandas as pd
DATA_WTI = pd.read_csv("../Datasets/Futures_Oil_WTI.csv", header=0)
DATA_BRENT = pd.read_csv("../Datasets/Futures_Oil_Brent.csv", header=0)
DATA_DUBAI = pd.read_csv("../Datasets/Futures_Oil_Dubai.csv", header=0)

In [7]:
import pandas as pd

# Load datasets
DATA_WTI = pd.read_csv("../Datasets/Futures_Oil_WTI.csv", header=0)
DATA_BRENT = pd.read_csv("../Datasets/Futures_Oil_Brent.csv", header=0)
DATA_DUBAI = pd.read_csv("../Datasets/Futures_Oil_Dubai.csv", header=0)

# Ensure the Date columns are parsed correctly
DATA_WTI['Date'] = pd.to_datetime(DATA_WTI['Date'], errors='coerce', format=r"%d-%m-%y")
DATA_BRENT['Date'] = pd.to_datetime(DATA_BRENT['Date'], errors='coerce', format=r"%d-%m-%y")
DATA_DUBAI['Date'] = pd.to_datetime(DATA_DUBAI['Date'], errors='coerce', format=r"%d-%m-%y")

# Perform full outer join on the Date column
combined_data = (
    DATA_WTI[['Date', 'Change %', 'Price', 'Vol.']]  # Include Price and Vol for WTI
    .rename(columns={'Change %': 'Change_WTI', 'Price': 'Price_WTI', 'Vol.': 'Vol_WTI'})
    .merge(
        DATA_BRENT[['Date', 'Change %', 'Price', 'Vol.']]
        .rename(columns={'Change %': 'Change_Brent', 'Price': 'Price_Brent', 'Vol.': 'Vol_Brent'}),
        on='Date',
        how='outer'
    )
    .merge(
        DATA_DUBAI[['Date', 'Change %', 'Price', 'Vol.']]
        .rename(columns={'Change %': 'Change_Dubai', 'Price': 'Price_Dubai', 'Vol.': 'Vol_Dubai'}),
        on='Date',
        how='outer'
    )
)

# Remove '%' from Change % columns and convert to numeric
for column in ['Change_WTI', 'Change_Brent', 'Change_Dubai']:
    combined_data[column] = (
        combined_data[column]
        .str.replace('%', '', regex=False)  # Remove '%' symbol
        .str.strip()  # Remove leading/trailing spaces
        .astype(float)  # Convert to numeric
        .divide(100)  # Convert to decimal
    )

# Sort by Date
combined_data = combined_data.sort_values(by='Date')

# Display the first few rows
display(combined_data)

# Save combined data to CSV if needed
combined_data.to_csv("../Datasets/Futures_Oil_Combined.csv", index=False)


,Date,Change_WTI,Price_WTI,Vol_WTI,Change_Brent,Price_Brent,Vol_Brent,Change_Dubai,Price_Dubai,Vol_Dubai
0,2000-01-04,-0.0020,25.55,74.79K,-0.0275,24.39,32.51K,NaN,NaN,NaN
1,2000-01-05,-0.0250,24.91,70.94K,-0.0271,23.73,30.31K,NaN,NaN,NaN
2,2000-01-06,-0.0052,24.78,85.25K,-0.0046,23.62,44.66K,NaN,NaN,NaN
3,2000-01-07,-0.0226,24.22,91.83K,-0.0224,23.09,34.83K,NaN,NaN,NaN
4,2000-01-10,0.0186,24.67,68.90K,0.0277,23.73,26.39K,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6478,2024-11-25,-0.0323,68.94,380.92K,-0.0217,73.01,282.16K,-0.0064,72.60,NaN
6479,2024-11-26,-0.0025,68.77,292.14K,-0.0027,72.81,200.70K,0.0000,72.60,NaN
6480,2024-11-27,-0.0007,68.72,227.22K,0.0003,72.83,106.53K,0.0000,72.60,NaN
6481,2024-11-28,NaN,NaN,NaN,0.0062,73.28,49.82K,NaN,NaN,NaN
